In [9]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchinfo import summary
import tqdm as notebook_tqdm
import os

import matplotlib.pyplot as plt
import numpy as np

print('pytorch version: ', torch.__version__)

pytorch version:  2.1.0


In [8]:
# cpu를 이용할지 gpu를 이용할지 설정
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('device: ', device)

device:  cpu


In [10]:
# 뒤에서 여러번 사용할 값들을 저장할 변수 +
# 모델에 설정할 값(모델자체, 학습할때 필요한 값 등등)으로 성능에 영향을 주는 값들을 저장할 변수 -> hyper parameter 

BATCH_SIZE = 256   # 모델의 파라미터를 업데이트할 때 사용할 데이터의 개수 (한 번에 몇 개의 데이터를 입력할지를 정의)
N_EPOCH = 20   # 전체 train dataset을 한 번 학습한 것을 1epoch
LR = 0.001    # 학습률: 파라미터 update할 때 gradient 값에 곱해줄 값 (새로운 parameter를 계산할 때, gradient를 얼마나 반영할지 비율)

# step : parameter를 한 번 업데이트하는 단위 (1 step 당 학습데이터수: batch size)
# epoch : 전체 학습데이터셋을 한 번 학습한 단위
#         1 epoch 당 step 횟수 = ceil(총 데이터 수 / batch size)

DATASET_SAVE_PATH = 'datasets'  # 데이터셋을 저장할 디렉토리 경로
MODEL_SAVE_PATH = 'models'  # 학습-평가가 끝난 모델을 저장할 디렉토리 경로

os.makedirs(DATASET_SAVE_PATH, exist_ok=True)
os.makedirs(MODEL_SAVE_PATH, exist_ok=True)

## MNIST dataset Loading

### Dataset

In [11]:
### MNIST Dataset을 다운로드 + dataset 객체를 생성
# tarin dataset
## torchvision.datasets
train_set = datasets.MNIST(root=DATASET_SAVE_PATH,  # dataset을 저장할 디렉도리 경로
                           train=True,  # trainset(훈련용): True, testset(검증용): False
                           download=True,  # root에 저장된 데이터파일들이 없을때 다운로드 받을지 여부
                           transform=transforms.ToTensor()   # 데이터 전처리
                          )
# ToTensor(): ndarray, PIL.Image 객체를 torch.Tensor로 변환.
#             pixcel값 정규화(normalize) - 0~1 사이의 실수로 변환. 
test_set = datasets.MNIST(root=DATASET_SAVE_PATH,
                          train=False,
                          download=True,
                          transform=transforms.ToTensor())

100%|███████████████████████████| 9912422/9912422 [00:00<00:00, 25337447.74it/s]


Extracting datasets/MNIST/raw/train-images-idx3-ubyte.gz to datasets/MNIST/raw



100%|███████████████████████████████| 28881/28881 [00:00<00:00, 36366164.46it/s]

Extracting datasets/MNIST/raw/train-labels-idx1-ubyte.gz to datasets/MNIST/raw




100%|███████████████████████████| 1648877/1648877 [00:00<00:00, 11530194.54it/s]


Extracting datasets/MNIST/raw/t10k-images-idx3-ubyte.gz to datasets/MNIST/raw



100%|██████████████████████████████████| 4542/4542 [00:00<00:00, 7766216.38it/s]

Extracting datasets/MNIST/raw/t10k-labels-idx1-ubyte.gz to datasets/MNIST/raw



In [12]:
type(train_set)   # Dataset type

torchvision.datasets.mnist.MNIST

In [13]:
print(train_set)

Dataset MNIST
    Number of datapoints: 60000
    Root location: datasets
    Split: Train
    StandardTransform
Transform: ToTensor()


In [14]:
print(test_set)

Dataset MNIST
    Number of datapoints: 10000
    Root location: datasets
    Split: Test
    StandardTransform
Transform: ToTensor()


In [15]:
# 데이터의 개수 확인
print(len(train_set), len(test_set))

60000 10000


### DataLoader

In [16]:
# Dataset을 모델에 어떻게 제공할지를 설정 -> 학습/평가시 설정된대로 데이터를 loading
# 학습훈련용 Dataloader
train_loader = DataLoader(train_set,   # Dataset
                          batch_size=BATCH_SIZE,   # batch_size를 설정
                          shuffle=True,    # 한 epoch이 끝나면 다음 epoch 전에 데이터를 섞을지 여부를 결정함
                          drop_last=True,  # 마지막 batch의 데이터 수가 설정된 batch_size보다 적을 경우 drop(학습에 사용x)할지 여부
                         )
# 평가용 Dataloader
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE)

In [17]:
print('1 epoch당 step 수')
print('trainset: ', len(train_loader))
print('testset: ', len(test_loader))

1 epoch당 step 수
trainset:  234
testset:  40


## 네트워크(모델) 정의
- Network : 전체 모델 구조

In [32]:
# class로 정의 : nn.Module을 상속해서 만듦
class MNistModel(nn.Module):

    def __init__(self):
        """
        모델 객체 생성시 , 모델구현(정의)에 필요한 것들을 초기화
        필요한 것: layer 등등.
        """
        super().__init__()

        ## 783(pixcel수) -> 128개로 축소
        self.lr1 = nn.Linear(28*28, 128)   # input feature의 크기 지정: 784개의 feature를 입력받음, output size: 128
        
        ## 128 feature -> 64개로 축소
        self.lr2 = nn.Linear(128, 64)
        
        ## 64 feature -> 출력결과 10(각 범주의 확률)
        self.lr3 = nn.Linear(64, 10)

        ## activation function define (ReLU 사용)
        self.relu = nn.ReLU()    # f(x) = max(x,0).  음수는 무조건 0으로 return 된다.
        

    def forward(self, x):
        """
        input data를 입력받아서 output data를 만들때 까지의 계산 흐름을 정의
        ===> foward propagation
        parameter
            x: input data
        return 
            torch.Tensor: output data(prediction result)
        """
        # init에서 생성한 function들을 이용해서 계산
        ## 순서 :  x -> 1차원으로 변환 -> lr1 -> ReLU -> lr2 -> ReLU -> lr3 -> output 
        x = torch.flatten(x, start_dim=1)  # input(batch_size, channel, height, width) -> (batch_size, 전체pixcel)로 flatten 시키기
        
        x = self.lr1(x)
        x = self.relu(x)
        
        x = self.lr2(x)
        x = self.relu(x)
        
        output = self.lr3(x)
        return output

In [20]:
i = torch.arange(28*28).reshape(1, 1, 28, 28)
print(i.shape) # [데이터개수, 채널, height, width]

# 데이터 개수는 유지하면서 [1, 784]로 바꾸기. 
# 다차원 tensor -> 1차원 : flatten([start_dim=axis]) 사용. 지정한 axis부터 합친다
torch.flatten(i, start_dim=1).shape  # axis0은 유지하고, axis1부터를 합친다(flatten).

torch.Size([1, 1, 28, 28])


torch.Size([1, 784])

In [35]:
# 정의한 모델 클래스로부터 모덷ㄹ 객체를 생성
model = MNistModel()
print(model)

MNistModel(
  (lr1): Linear(in_features=784, out_features=128, bias=True)
  (lr2): Linear(in_features=128, out_features=64, bias=True)
  (lr3): Linear(in_features=64, out_features=10, bias=True)
  (relu): ReLU()
)


In [37]:
# model의 연산 흐름 및 정보를 확인 => torchinfo 패키지를 사용
torchinfo.summary(model, (256, 1, 28, 28)) # summary(정보를 확인하기위한 모델 객체, input shape; tuple)

Layer (type:depth-idx)                   Output Shape              Param #
MNistModel                               [256, 10]                 --
├─Linear: 1-1                            [256, 128]                100,480
├─ReLU: 1-2                              [256, 128]                --
├─Linear: 1-3                            [256, 64]                 8,256
├─ReLU: 1-4                              [256, 64]                 --
├─Linear: 1-5                            [256, 10]                 650
Total params: 109,386
Trainable params: 109,386
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 28.00
Input size (MB): 0.80
Forward/backward pass size (MB): 0.41
Params size (MB): 0.44
Estimated Total Size (MB): 1.65

In [39]:
# train dataset의 첫 번째 batch를 이용해서 inference 해보자.
x_batch, y_batch = next(iter(train_loader))   # train_loader의 첫 번째 반복값을 가져온다. 튜플로 묶어서 가져와서 두 개의 변수로 따로따로 줌.
print(x_batch.shape, y_batch.shape)

torch.Size([256, 1, 28, 28]) torch.Size([256])


In [42]:
pred_batch = model(x_batch)  # model의 forward() 메소드가 실행됨
pred_batch.shape  # 지정해준 크기의 output이 나온 것 확인

torch.Size([256, 10])

In [43]:
pred_batch[0]  # 첫 번째 이미지에 대한 추론결과 (class별 확률 도출! 가장 큰 확률의 index를 조회해 보자.)

tensor([-0.2097,  0.1269,  0.0903,  0.0011, -0.0487, -0.0156, -0.0315, -0.0427,
         0.1150, -0.1054], grad_fn=<SelectBackward0>)

In [44]:
# 1 index가 제일 크니까, 1이미지로 예측한 것
pred_batch[0].argmax()   # 1 index가 제일 크니까, 1이미지로 예측한 것

tensor(1)

In [45]:
y_batch[0]   # 정답은 6이미지 이었음 -> 예측이 틀림!

tensor(6)

In [47]:
# pred_batch의 예측값과 y_batch가 동일한 것의 개수는? (즉, 맞은 개수는?)
torch.sum(pred_batch.argmax(dim=1) == y_batch)

tensor(22)